3.11.11 (main, Mar 17 2025, 21:01:29) [MSC v.1943 64 bit (AMD64)]


In [2]:
# 安装依赖
%uv add --upgrade langchain-text-splitters langchain-community langgraph

Note: you may need to restart the kernel to use updated packages.


Resolved 156 packages in 3.26s
Audited 151 packages in 1ms


In [ ]:
# 相关环境变量设置
import sys 
sys.path.append("..") 
from config import config_loader

config_loader.load_env()

In [4]:
# 定义 chat 模型类型
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# 测试模型连通性
llm.invoke("你好！")

AIMessage(content='你好！很高兴能和你交流。有什么我可以帮你的吗？', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-17a04aae-aa6a-4e23-ad6d-31716e9b5c99-0', usage_metadata={'input_tokens': 2, 'output_tokens': 15, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})

In [5]:
# 定义嵌入模型
from langchain_google_genai import GoogleGenerativeAIEmbeddings

emmbeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

# 测试
emmbeddings.embed_query("你好！")

[-0.024826884269714355,
 0.018420174717903137,
 -0.03160383179783821,
 -0.004086533561348915,
 0.03807083144783974,
 0.015953589230775833,
 0.041565556079149246,
 -0.018995137885212898,
 0.04406355693936348,
 0.01584736630320549,
 -0.05369194597005844,
 0.0399884432554245,
 0.04639393836259842,
 -0.012052264995872974,
 -0.004960070364177227,
 -0.06393793225288391,
 -0.008106617256999016,
 0.0005135182873345912,
 -0.09638288617134094,
 0.024841194972395897,
 0.053779780864715576,
 0.00759957917034626,
 0.005360820330679417,
 0.005029894877225161,
 -0.0036309980787336826,
 0.0008750242996029556,
 0.005991972982883453,
 0.02845136821269989,
 -0.03126361221075058,
 -0.030745001509785652,
 -0.027942778542637825,
 0.0642596185207367,
 0.008675439283251762,
 -0.04587496444582939,
 0.023483358323574066,
 0.02252344787120819,
 0.007605357561260462,
 0.07887224853038788,
 0.03535320237278938,
 -0.097498320043087,
 -0.0663716271519661,
 0.011278927326202393,
 -0.03863407298922539,
 0.034613210707

In [8]:
# 定义向量存储 使用内存存储
from langchain_core.vectorstores import InMemoryVectorStore

# 要传入嵌入模型对象，后面就用向量存储对象来操作
vector_store = InMemoryVectorStore(emmbeddings)

vector_store.add_texts("我的名字是张三")

['d376ff85-ff67-46b7-8d2a-584fa8b47180',
 '47c84635-0dec-4098-a2f0-5389c08681cb',
 'e637c1f8-008d-4cdf-bd30-5e515392c65c',
 '20eb12dc-39a8-4608-9cac-ed15d55ee218',
 '8b309e58-3270-4905-9d22-8da09c208cc7',
 '1b148dfc-0615-4919-a20e-76815d4f08a9',
 'ca84ce7c-bee0-48a5-90bc-bae25316411b']

In [13]:
# 数据准备，从外部网站上获取数据
from langchain_community.document_loaders import WebBaseLoader
import bs4
import os

# 必须得设置这个环境变量 不然不能运行
os.environ['USER_AGENT'] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36"

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
# 返回 list[Document]
docs = loader.load()
print(f'docs len: {len(docs)}')
docs

docs len: 1


[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [14]:
# 爬取下来的是一整个文档，有很多字，所以需要分段 使用通用的文本分割器
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 定义文本分割器，分割成 1000 字的片段，重叠 200 字，add_start_index 表示是在每个片段的开头加上索引元信息
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)

# 文本分段
all_splits = text_splitter.split_documents(docs)

print(f'文本被分成了 {len(all_splits)} 个片段，第一个分段信息如下：')
all_splits[0]

文本被分成了 66 个片段，第一个分段信息如下：


Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 8}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from 

In [17]:
# 索引入库
idx_result = vector_store.add_documents(all_splits)

print(f'索引结果：{len(idx_result)}')
print(f'索引结果第一个元素：{idx_result[0]}')

索引结果：66
索引结果第一个元素：457ca4c1-a09e-4c85-b474-a1ddc0ea0312


In [19]:
# 定义 prompt 模板 直接拿别人写好的 rag-prompt
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
# 定义应用的State对象，这个是 LangGraph 中节点之间的传参的参数类型
from typing import List, TypedDict
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph

class State(TypedDict):
    question: str
    answer: str
    # 相似分段信息
    context: List[Document]

# 定义应用的步骤
# 检索函数
def retrieve(state: State):
    related_docs = vector_store.similarity_search(state["question"])
    return {"context": related_docs}

def generate(state: State):
    # 相似文档转换为字符串
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({
        "question": state["question"],
        "context": docs_content
    })
    response = llm.invoke(messages)
    return {"answer": response}

# 规划 graph 并编译
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()


In [22]:
# 使用
response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])

content='Task decomposition is a method of breaking down complex tasks into smaller, simpler steps. This is often achieved using techniques like Chain of Thought (CoT), which instructs the model to "think step by step". Task decomposition can be done by LLMs, task-specific instructions or human inputs.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-4f21907b-0c2c-4a92-8d5f-430994c6090e-0' usage_metadata={'input_tokens': 611, 'output_tokens': 59, 'total_tokens': 670, 'input_token_details': {'cache_read': 0}}
